In [1]:
# import block
import numpy as np
from astropy.io import fits
%matplotlib inline
from matplotlib import pyplot as plt
from astropy.visualization import hist
import ccdproc as ccdp
from matplotlib import rc
from datetime import datetime
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D, Gaussian1D
from astropy.modeling import fitting
from astropy.stats import mad_std
from astropy.nddata import CCDData
from scipy import stats
from astropy import units as u

In [2]:
# import convenience plotting functions downloaded from 
# here: https://github.com/mwcraig/ccd-reduction-and-photometry-guide
phot_tutorial_dir = '/home/kyle/Homework/Astro8060/ccd-reduction-and-photometry-guide/notebooks/'
import sys
sys.path.insert(0,phot_tutorial_dir)
from convenience_functions import show_image

In [3]:
# function for flat scaling
def inv_median(x):
    return 1./np.median(x)

In [4]:
data_dir = '/mnt/chromeos/removable/ChromeSD/NoBackupImages/raw/'            # raw data directory
reduced_dir = '/mnt/chromeos/removable/ChromeSD/NoBackupImages/reduced/' # working directory

## Workspace and data preparation

Identify the data and reduced directories. These will need to be set by computer. The data_dir variable should be set to a directory containing the raw data. The reduced_dir variable should be set to an empty directory where the reduced data will be stored.

Copy over relevant image files and update their header keywords. This step is messy because the files for different types of frames are all over the place in the log.txt file. The approach here is to generate arrays of filenames by counting iteratively from the lowest to highest number relevant for a particular file type, then delete any unnecessary numbers from the list. The filters are hardcoded in to assign the correct filter to the correct filename, based on information in the log. An alternative (and possibly cleaner) to this approach would be to store lists of the relevant file types or filters.

In [ ]:
# read in the biases, they're in a093-a111.fits
# add some keywords to their headers and save
# them to the reduced/ directory
bias_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(19)+93]
hdus = [fits.open(data_dir+file) for file in bias_files]
hdrs = [hdu[0].header for hdu in hdus]
    
for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'bias'
    hdrs[i]['object']   = 'none'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS indexing, row/col reversed
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    hdrs[i]['filter']   = 'none'
    
    hdus[i].writeto(reduced_dir+bias_files[i],overwrite=True)
    
del hdus
del hdrs

In [ ]:
# read in the flats, they're in a005-a062.fits
# add some keywords to their headers and save
# them to the reduced/ directory
flat_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(58)+5]

# delete saturated/bad flats from the list
del flat_files[16], flat_files[23:26], flat_files[47]

hdus = [fits.open(data_dir+file) for file in flat_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'skyflat'
    hdrs[i]['object']   = 'none'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]'# FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=16) & (i<=22)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=22) & (i<= 33)):
        hdrs[i]['filter']   = 'B'
    elif ((i>=33) & (i<= 40)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=40) & (i<= 46)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=46) & (i<= 52)):
        hdrs[i]['filter']   = 'I'  
    else:
        hdrs[i]['filter']   = 'Ha'
    
    hdus[i].writeto(reduced_dir+flat_files[i],overwrite=True)

del hdus
del hdrs

In [ ]:
# now do the darks
dark_files = ['d'+str(i).zfill(3)+'.fits' for i in np.arange(15)+1]
hdus = [fits.open(data_dir+file) for file in dark_files]
hdrs = [hdu[0].header for hdu in hdus]
    
for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'dark'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    hdrs[i]['filter']   = 'none'
    
    hdus[i].writeto(reduced_dir+dark_files[i],overwrite=True)
    
del hdus
del hdrs

In [ ]:
# now do the pg1633 science images, they're all over the place so generate the full
# range and then delete the intervals between exposure sets
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(173)+74]
del science_files[19:56]
del science_files[29:91]
del science_files[39:64]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'pg1633'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=7) & (i<=8)) | ((i>=19) & (i<=20)) | ((i>=29) & (i<=30)) | ((i>=39) & (i<=40)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=8) & (i<=10)) | ((i>=20) & (i<=22)) | ((i>=30) & (i<=32)) | ((i>=40) & (i<=42)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=0) & (i<=6)) | ((i>=10) & (i<=12)) | ((i>=22) & (i<=24)) | ((i>=32) & (i<=34)) | ((i>=42) & (i<=44)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=12) & (i<=14)) | ((i>=24) & (i<=26)) | ((i>=34) & (i<=36)) | ((i>=44) & (i<=46)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=14) & (i<=16)) | ((i>=26) & (i<=28)) | ((i>=36) & (i<=38)) | ((i>=46) & (i<=48)):
        hdrs[i]['filter']   = 'I'  
    else:
        hdrs[i]['filter']   = 'Ha'
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

In [ ]:
# now do the other ngc6823 science images, they're in two sets
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(11)+153]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'ngc6823'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

## Overscan subtraction, trim, and master bias subtraction

Do overscan substraction, trim the images, and save with *ot.fits filenames. Then make a master bias using an average combine and 3 sigma clipping. Subtract the master bias from all frames and save with *otz.fits flags.

In [5]:
# make an image collection 
imgs = ImageFileCollection(reduced_dir,glob_include='[ad]???.fits',glob_exclude='*ot*.fits') # exclude master*, otz files in case of rerun

In [6]:
import warnings
warnings.filterwarnings('ignore')

# set a model to characterize overscan columns
model = Polynomial1D(3)

# loop over all frames and do subtraction and trim
for ccd, file_name in imgs.ccds(ccd_kwargs={'unit': 'adu'},return_fname=True):
    print(file_name)
    try:
        ccd = ccdp.subtract_overscan(ccd, fits_section=ccd.header['biassec'], median=False, model=model)  # subtract overscan, fits index
        ccd = ccdp.trim_image(ccd,fits_section=ccd.header['trimsec']) # trim images
        outfile = file_name.strip('.fits')+'ot.fits'
        ccd.write(reduced_dir+'/'+outfile, overwrite=True) # save the result
    except Exception as e:
        print(e)
        print('Could not trim and subtract: '+file_name)
        

a001.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a001.fits
a002.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a002.fits
a003.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a003.fits
a004.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a004.fits
a005.fits
a006.fits
a007.fits
a008.fits
a009.fits
a010.fits
a011.fits
a012.fits
a013.fits
a014.fits
a015.fits
a016.fits
a017.fits
a018.fits
a019.fits
a020.fits
a021.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a021.fits
a022.fits
a023.fits
a024.fits
a025.fits
a026.fits
a027.fits
a028.fits
a029.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a029.fits
a030.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a030.fits
a031.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a031.fits
a032.fits
a033.fits
a034.fits
a035.fits
a036.fits
a037.fits
a038.fits
a039.fits
a040.fits
a041.fits
a042.fits
a043.fits
a044.fits


a217.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a217.fits
a218.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a218.fits
a219.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a219.fits
a220.fits
"Keyword 'BIASSEC' not found."
Could not trim and subtract: a220.fits
a237.fits
a238.fits
a239.fits
a240.fits
a241.fits
a242.fits
a243.fits
a244.fits
a245.fits
a246.fits
d001.fits
d002.fits
d003.fits
d004.fits
d005.fits
d006.fits
d007.fits
d008.fits
d009.fits
d010.fits
d011.fits
d012.fits
d013.fits
d014.fits
d015.fits


In [7]:
# make a master bias from overscan subtracted, trimmed frames
# this does a mean combined with 3 sigma clipping
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='[ad]???ot.fits',glob_exclude='master*')
biases = imgs.files_filtered(imagetyp='bias', include_path=True)

master_bias = ccdp.combine(biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=3, sigma_clip_high_thresh=3,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             mem_limit=350e6, unit=u.adu
                            )

master_bias.meta['combined'] = True
master_bias.write(reduced_dir+'master_bias.fits',overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [8]:
# subtract the master bias from all frames
for ccd, file_name in imgs.ccds(ccd_kwargs={'unit': 'adu'},return_fname=True):
    frame_nobias = ccdp.subtract_bias(ccd, master_bias)
    outfile = file_name.strip('.fits')+'tz.fits'
    frame_nobias.write(reduced_dir+'/'+outfile, overwrite=True) # save the result   

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


## Do the flat fielding

Start with the overscan subtracted, trimmed, master bias subtracted frames and identify all the sky flats. Loop over the bands and combine them. Do a median scaled median combine with 2 sigma clipping (other options, e.g. including weighting, may be more ideal).

In [9]:
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='[ad]*otz.fits',glob_exclude='master*') # just read in the bias-subtracted images
flats = imgs.summary['imagetyp'] == 'skyflat'
flat_filters = set(imgs.summary['filter'][flats])

for filtr in flat_filters:
    flat_infilter = imgs.files_filtered(imagetyp='skyflat', filter=filtr,include_path=True)
    
    # do a median combine with 2 sigma clipping
    master_flat = ccdp.combine(flat_infilter,
                             method='median', scale=inv_median,
                             sigma_clip=True, sigma_clip_low_thresh=2, sigma_clip_high_thresh=2,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                             mem_limit=350e6
                            ) 


    master_flat.meta['combined'] = True
    flat_file_name = 'master_flat_filter_{}.fits'.format(filtr.replace("''", "p"))
    master_flat.write(reduced_dir+flat_file_name,overwrite=True)

INFO:astropy:splitting each image into 5 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 5 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 9 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 9 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 5 chunks to limit memory usage to 350000000.0 bytes.


...

INFO: splitting each image into 6 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 6 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 6 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 13 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 13 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


Normalize the master flats. This may not be technically necessary, because CCDPROC says that it scales the flat by its mean value before dividing it out in ccdp.correct_flat().

In [10]:
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='master_flat_filter*.fits')
flat_files = imgs.files_filtered(imagetyp='skyflat',include_path=True)
bands = imgs.filter(imagetyp='skyflat').summary['filter'].data.data

for ccd in imgs.ccds(imagetyp='skyflat', combined=True):
    band = ccd.meta['filter']
    
    scale = np.mean(ccd.data)
    ccd.data = ccd.data/scale
        
    flat_file_name = 'master_flat_norm_filter_{}.fits'.format(band.replace("''", "p"))
    ccd.write(reduced_dir+flat_file_name,overwrite=True)

Flatfield the reduced science frames and save them with the *otzf.fits flags.

In [11]:
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='[ad]???otz.fits',glob_exclude='master*.fits') # all biases, darks, raw flats, science frames
flats        = ccdp.ImageFileCollection(reduced_dir,glob_include='master*norm*.fits')
sci_frames   = {ccd.header['filter']: ccd for ccd in imgs.ccds(imagetyp='science')} # only flatfield the science frames
master_flats = {ccd.header['filter']: ccd for ccd in flats.ccds(imagetyp='skyflat', combined=True)}

In [13]:
for ccd, file_name in imgs.ccds(imagetyp='science', return_fname=True, ccd_kwargs=dict(unit='adu')):
    flat_in_filter = master_flats[ccd.header['filter']]
    reduced = ccdp.flat_correct(ccd, flat_in_filter)
    outfile = file_name.strip('otz.fits')+'otzf.fits'
    
    reduced.write(reduced_dir+outfile,overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
